# Importing libraries

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch
import torchsummary
from torchsummary import summary

In [ ]:
testing_set = torch.load('server_t.pth')
train_data = torch.load('server.pth')

In [ ]:
class autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  
            nn.Conv2d(16, 8, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2), 
            
            nn.Conv2d(8, 4, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2), 
            
        )
        self.decoder = nn.Sequential(
            
            nn.ConvTranspose2d(4, 8, 3, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(8, 16, 2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 32, 2, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 1, 3, padding=1) 
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        x = torch.sigmoid(x)  
        return x
model = autoencoder()
print(summary(model,(1,28,28)))

In [ ]:
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())
epochs = 200
outputs = []
losses = []
for e in range(epochs): 
    print(e)
    total_loss = 0
    for idx, (features,labels) in enumerate(train_data):
        predictions= model(features)
        loss = loss_function(predictions, features)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss)
    outputs.append((epochs, features, predictions)) 

In [ ]:
print(features[0].size(), predictions[0].size())

In [ ]:
plt.imshow(predictions[71].detach().numpy().reshape(28,28))

In [ ]:
plt.imshow(features[71].detach().numpy().reshape(28,28))

In [ ]:
torch.save(model, 'auto_encoder.pth')

This is the code of an autoencoder model and we save this model,so that for each client and each case we can use the same model parameters. This model is used by the server to reduce the dimension of the MNIST feature which is an image of 28*28 pixels.  